## Hopfield Modell

zunächst werdenen die benötigten Bibliotheken implementiert

In [6]:
import numpy as np
import cv2
import tkinter as tk
import matplotlib.pyplot as plt
import tkinter.filedialog
import matplotlib
import threading
matplotlib.use('TkAgg')

Das hier ist lediglich eine funktion um später anschaulicher zu plotten

In [2]:
def plot_params(training_patterns, testing_pattern):
    num_training_patterns = len(training_patterns)
    fig1, axs1 = plt.subplots(1, num_training_patterns + 1, figsize=(num_training_patterns * 4, 4))
    
    for i, pattern in enumerate(training_patterns):
        axs1[i].imshow(pattern, cmap='gray')
        axs1[i].set_title(f'Training Pattern {i+1}')
    
    axs1[num_training_patterns].imshow(testing_pattern[0], cmap='gray')
    axs1[num_training_patterns].set_title('Testing Pattern')
    fig1.suptitle('Netzwerkparamter')
    plt.tight_layout()


Hier wird das Programm zur Pixelauswahl Definiert

In [3]:
class PixelGridApp:
    def __init__(self, master, mode):
        self.master = master
        self.master.title("Hopfield Modell Musterauswahl")
        self.grid_size = 5
        self.cell_size = 40 # Jede Zelle ist 40 X 40 Pixel Groß
        self.patterns = []
        self.mode = mode # Modus zur bestimmung ob man im testing oder im trainig modus ist.

        self.grid = np.full((self.grid_size, self.grid_size), 1, dtype=int)
        self.canvases = []

        self.create_widgets()
        self.update_mode_label()
    
    # Im folgenden werden die ganzen Bausteinen und Knöppfe definiert. Den Knöpfen wird jeweils eine Funktion zugeteil die weiter unten definiert sind.
    def create_widgets(self):
        for i in range(self.grid_size):
            row_canvases = []
            for j in range(self.grid_size):
                canvas = tk.Canvas(self.master, width=self.cell_size, height=self.cell_size, bg="white")
                canvas.grid(row=i, column=j)
                canvas.bind("<Button-1>", lambda event, i=i, j=j: self.toggle_pixel(i, j))
                canvas.create_text(self.cell_size//2, self.cell_size//2, fill="black", font=("Helvetica", 16))
                row_canvases.append(canvas)
            self.canvases.append(row_canvases)

        self.save_button = tk.Button(self.master, text="Save Pattern", command=self.save_pattern)
        self.save_button.grid(row=self.grid_size, column=0, columnspan=self.grid_size // 2)

        self.reset_button = tk.Button(self.master, text="Reset Grid", command=self.reset_grid)
        self.reset_button.grid(row=self.grid_size, column=self.grid_size // 2, columnspan=self.grid_size // 2)

        self.save_file_button = tk.Button(self.master, text="Save to File", command=self.prompt_save_file)
        self.save_file_button.grid(row=self.grid_size + 1, column=0, columnspan=self.grid_size // 2)

        self.load_file_button = tk.Button(self.master, text="Load from File", command=self.prompt_load_file)
        self.load_file_button.grid(row=self.grid_size + 1, column=self.grid_size // 2, columnspan=self.grid_size // 2)

        self.finish_button = tk.Button(self.master, text="Finish", command=self.finish)
        self.finish_button.grid(row=self.grid_size + 2, column=0, columnspan=self.grid_size)

        self.mode_label = tk.Label(self.master, text="")
        self.mode_label.grid(row=self.grid_size + 3, column=0, columnspan=self.grid_size)
    
    def update_mode_label(self):
        if self.mode == "training":
            self.mode_label.config(text="Mode: Training")
        elif self.mode == "testing":
            self.mode_label.config(text="Mode: Testing")

    def toggle_pixel(self, i, j):
        if self.grid[i, j] == 1:
            self.grid[i, j] = -1
            self.canvases[i][j].config(bg="black")
            self.canvases[i][j].itemconfig(1, fill="white")
        else:
            self.grid[i, j] = 1
            self.canvases[i][j].config(bg="white")
            self.canvases[i][j].itemconfig(1, fill="black")
    
    def save_pattern(self):
        self.patterns.append(self.grid.copy())
        print("Pattern saved:")
        print(self.grid)
        if self.mode == 'testing':
            self.testing_mode()
    
    def reset_grid(self):
        self.grid.fill(1)
        for row_canvases in self.canvases:
            for canvas in row_canvases:
                canvas.config(bg="white")
                canvas.itemconfig(1, fill="black")
        print("Grid reset")
    
    def prompt_save_file(self):
        filename = tk.filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
        if filename:
            self.save_patterns_to_file(filename)
            if self.mode == "training":
                self.switch_to_testing_mode()
            if self.mode == "testing":
                self.testing_mode()
    
    def save_patterns_to_file(self, filename):
        np.savetxt(filename, np.array(self.patterns).reshape(len(self.patterns), -1), fmt='%d')
        print(f"Patterns saved to {filename}")
    
    def prompt_load_file(self):
        filename = tk.filedialog.askopenfilename(filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
        if filename:
            self.load_patterns_from_file(filename)
            if self.mode == "training":
                self.switch_to_testing_mode()
            if self.mode == "testing":
                self.testing_mode()
            
    
    def load_patterns_from_file(self, filename):
        data = np.loadtxt(filename, dtype=int)
        if self.mode == "testing":
            self.patterns = data.reshape(1, self.grid_size, self.grid_size).tolist()
        else:
            num_patterns = data.shape[0]
            self.patterns = data.reshape(num_patterns, self.grid_size, self.grid_size).tolist()
        print(f"Patterns loaded from {filename}")
    
    def switch_to_testing_mode(self):
        self.training_patterns = np.array(self.patterns)
        self.mode = "testing"
        self.patterns = []
        self.update_mode_label()
        self.reset_grid()
        print("Switched to Testing mode")

    def testing_mode(self):
        self.testing_pattern = np.array(self.patterns)
        print('Testing Pattern saved in Array')

    
    def finish(self):
        try:
            self.testing_pattern
        except:
            self.testing_pattern = np.zeros((5, 5))
            print('Error: Nichtsts gespeichert')
        try:
            self.training_patterns
        except:
            self.training_patterns = np.zeros((5, 5))
            print('Error: Nichtsts gespeichert')   
        self.master.quit()  # Beendet die Tkinter-Mainloop
        self.master.destroy()  # Zerstört das Tkinter-Fenster


Jetzt wird zunächst das Hopfieldmodel implementiert

In [23]:
class HopfieldNetwork:
    
    #Initierungsfunktion
    def __init__(self, num_neurons, steps):
        self.num_neurons = num_neurons
        self.matrix_size = int(np.sqrt(self.num_neurons))
        self.weights = np.zeros((num_neurons, num_neurons))
        self.steps = steps
    #Trainig durch die Hebbsche Regel. Normiert auf die Anzahl der Neuronen. Input sind gegebene Patterns
    def train(self, patterns):
        for p in patterns:
            p = p.flatten()  # Ensuring pattern is a column vector
            if len(p) != self.num_neurons:
                raise ValueError(f"Pattern size {len(p)} does not match the number of neurons {self.num_neurons}.")
            self.weights += np.outer(p, p)
        self.weights /= self.num_neurons
        np.fill_diagonal(self.weights, 0)  # No self-connections
        

    def show_as_matrix(self,pattern):
        matrix = np.reshape(pattern, (self.matrix_size, self.matrix_size))
        return matrix

    def predict(self, pattern, activation_function, axs = None):
        s = pattern.flatten()  # Kopie des Musters und Umformung zu Spaltenvektor
        for i in range(self.steps):
            a = np.dot(self.weights, s)
            s = activation_function(a)
            s = s.flatten()
            if axs is not None:
                axs[i].imshow(self.show_as_matrix(s),cmap='gray')
                axs[i].set_title(f'Nach {i+1} Steps')
                plt.tight_layout()
        return s.flatten()  # Rückgabe als flaches Array
    
    def predict2(self,pattern,activation_function):
        s = pattern.flatten()
        a = np.dot(self.weights,s)
        s = activation_function(a)
        return s
    

    def energy(self, pattern):
       
        return -0.5 * np.dot(np.dot(pattern.T, self.weights), pattern)
    
def tanh(a):
    return np.tanh(a)
    
def heviside(a):
    return np.where(a >= 0, 1, -1)




Rauschfunktion

In [8]:
def flip_values(matrix, probability):
    flip_mask = np.random.rand(*matrix.shape) < probability
    matrix[flip_mask] *= -1
    return matrix

In [6]:
# Beispielmuster
patterns = [np.array([1, -1, 1, -1]), np.array([-1, 1, 1, -1])]

# Hopfield-Netzwerk mit 4 Neuronen initialisieren
network = HopfieldNetwork(num_neurons=4, steps= 5)

# Netzwerk mit den Mustern trainieren
network.train(patterns)

# Vorhersage für ein verrauschtes Muster
test_pattern = np.array([-1, -1, 1, -1])
result = network.predict(test_pattern, heviside)

print("Ursprüngliches Muster:", test_pattern)
print("Erkanntes Muster:", result)
print("Energie des erkannten Musters:", network.energy(result))
matrix_result = network.show_as_matrix(result)
print(matrix_result)
plt.imshow(network.show_as_matrix(patterns[0]), cmap='gray', interpolation='none')
plt.show()
plt.imshow(network.show_as_matrix(patterns[1]), cmap='gray', interpolation='none')
plt.show()
plt.imshow(network.show_as_matrix(test_pattern), cmap='gray', interpolation='none')
plt.show()
plt.imshow(matrix_result, cmap='gray', interpolation='none')
plt.show()


Ursprüngliches Muster: [-1 -1  1 -1]
Erkanntes Muster: [ 1  1  1 -1]
Energie des erkannten Musters: -0.0
[[ 1  1]
 [ 1 -1]]


In [7]:
# Beispielmuster
patterns = [
    np.array([
        [1, 1, 1, 1, 1],
        [1, -1, 1, -1, 1],
        [1, 1, 1, 1, 1],
        [-1, 1, 1, 1, -1],
        [1, -1, -1, -1, 1]
    ]),
    np.array([
        [1, 1, 1, 1, 1],
        [1, -1, 1, -1, 1],
        [1, 1, 1, 1, 1],
        [1, -1, -1, -1, 1],
        [-1, 1, 1, 1, -1]
    ]),
    np.array([
        [1, 1, -1, 1, 1],
        [1, 1, -1, 1, 1],
        [1, 1, -1, 1, 1],
        [1, -1, 1, -1, 1],
        [-1, -1, 1, -1, -1]
    ])
]

# Hopfield-Netzwerk mit Neuronen initialisieren
network = HopfieldNetwork(num_neurons=25, steps= 2)

# Netzwerk mit den Mustern trainieren
network.train(patterns)

# Vorhersage für ein verrauschtes Muster
test_pattern =[
    np.array([
        [1, 1, -1, 1, 1],
        [1, -1, -1, -1, 1],
        [1, 1, -1, 1, 1], #Misichzustand
        [1, 1, -1, -1, 1],
        [-1, -1, 1, 1, -1]
    ]),
    np.array([
        [1, 1, -1, 1, 1],
        [1, -1, -1, -1, 1],
        [1, 1, -1, 1, 1],
        [1, -1, -1, -1, 1],
        [-1, 1, 1, 1, -1]
    ]),
        np.array([
        [1, 1, -1, 1, 1],
        [1, -1, -1, -1, 1],
        [1, 1, -1, 1, 1], # konvergiert zu rakete in 2 schritten
        [1, -1, 1, -1, 1],
        [-1, -1, 1, -1, -1]
    ])
]



# Zu beobachtendes pattern auswählen:
view_patter = 1
result = network.predict(test_pattern[view_patter],heviside)

print("Ursprüngliches Muster:", test_pattern[view_patter])
print("Erkanntes Muster:", result)
print("Energie des erkannten Musters:", network.energy(result))
matrix_result = network.show_as_matrix(result)
plt.imshow(network.show_as_matrix(patterns[0]), cmap='gray', interpolation='none')
plt.show()
plt.imshow(network.show_as_matrix(test_pattern[view_patter]), cmap='gray', interpolation='none')
plt.show()
plt.imshow(matrix_result, cmap='gray', interpolation='none')
plt.show()


Ursprüngliches Muster: [[ 1  1 -1  1  1]
 [ 1 -1 -1 -1  1]
 [ 1  1 -1  1  1]
 [ 1 -1 -1 -1  1]
 [-1  1  1  1 -1]]
Erkanntes Muster: [ 1  1  1  1  1  1 -1  1 -1  1  1  1  1  1  1  1 -1 -1 -1  1 -1  1  1  1
 -1]
Energie des erkannten Musters: -13.12


In [12]:
def capture_image():
    # Open the webcam (0 is the default device)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return None

    # Capture a single frame
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("Error: Could not capture image")
        return None

    # Convert the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray

def crop_center(image, crop_size):
    """
    Schneidet einen zentralen Ausschnitt aus einem Bild.

    Parameters:
    image (numpy.ndarray): Das Eingabebild als NumPy-Array.
    crop_size (int): Die Größe des quadratischen Ausschnitts.

    Returns:
    numpy.ndarray: Der zentrale Ausschnitt des Bildes.
    """
    h, w = image.shape
    startx = w // 2 - (crop_size // 2)
    starty = h // 2 - (crop_size // 2)
    return image[starty:starty + crop_size, startx:startx + crop_size]



def edit_image(gray_image,matrix_size):
    if gray_image is not None:
        img = crop_center(gray_image,700)
        img = np.array(cv2.resize(img, (matrix_size,matrix_size)))
        mean = np.mean(img)
        img = np.where( img >= mean, 1, -1)
        return img
        
    else:
        print("No image captured.")


#Hopfield Network starten mit gewünschten Parameter
network = HopfieldNetwork(num_neurons= 10000, steps= 5)
steps = network.steps 
matrix_size = network.matrix_size  
print(matrix_size)     
#Bildaxe und Plot initalisieren
fig1, axs1 = plt.subplots(1, 2)
fig2, axs2 = plt.subplots(1, steps)

# Bild Aufnehmen und lernen
gray_image = capture_image()
learn_image = edit_image(gray_image, matrix_size)
print(matrix_size)
axs1[0].imshow(learn_image)
axs1[0].set_title('Aufgenommenes Bild')
network.train([learn_image])
print(learn_image)

#Rauschen drautftun

noise_level = 0.5
noise_img = flip_values(learn_image,noise_level)

axs1[1].imshow(noise_img)
axs1[1].set_title(f'Bild mit Einem Noiselevel von {noise_level*100}% ')

fig1.suptitle('Netzwerkparamter')

#Netzwerk Anwenden

result = network.predict(noise_img,heviside, axs=axs2)
fig2.suptitle('Netzwerkausgabe')
plt.show()
#print(network.weights)



100
100
[[ 1 -1 -1 ...  1  1  1]
 [-1 -1 -1 ...  1 -1  1]
 [-1  1 -1 ...  1  1  1]
 ...
 [ 1  1  1 ... -1 -1 -1]
 [ 1  1 -1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]]


In [30]:
plt.close('all')
root = tk.Tk()
app = PixelGridApp(root, "training")
root.mainloop()


training = app.training_patterns
testing = app.testing_pattern


network = HopfieldNetwork(num_neurons=25, steps=5)
steps = network.steps

network.train(training)
plot_params(training_patterns=training, testing_pattern=testing)
fig2, axs2 = plt.subplots(1, steps, figsize=(len(training) * 4, 4))
result = network.predict(testing[0],np.sign, axs=axs2)
fig2.suptitle('Netzwerkausgabe')

plt.show()



Patterns loaded from /Users/phileasmeier/Documents/UNI/Neurale_Netzwerke/Hopfield_Model/Hopefield_Model/symbols_number_training.txt
Grid reset
Switched to Testing mode
Testing Pattern saved in Array
Pattern saved:
[[ 1 -1  1 -1  1]
 [-1  1 -1  1 -1]
 [-1  1  1  1 -1]
 [ 1  1  1 -1  1]
 [ 1  1 -1  1  1]]
Testing Pattern saved in Array
